# Joining consumotion and weather data

## Import the data from there files

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
from scipy.spatial import cKDTree
import numpy as np

In [4]:

# Specify the path to your CSV files
weather_csv_path = "/Users/computer/Downloads/Load/Load/data/weather_data/*.csv"

# Get all CSV file paths
weather_csv_files = glob.glob(weather_csv_path)

# Read and combine all files into one DataFrame
weather_df = [pd.read_csv(file, parse_dates=['date_time_weather']) for file in weather_csv_files]

# Merge dataframes of all processed files
weather_df = pd.concat(weather_df, ignore_index=True)

# Display the combined DataFrame
print(weather_df)


         Unnamed: 0.1   date_time_weather  Unnamed: 0    id         name  \
0                   0 2021-01-01 00:00:00   2392917.0   2.0     Al Jafer   
1                   1 2021-01-01 00:05:00   2392918.0   2.0     Al Jafer   
2                   2 2021-01-01 00:10:00   2392919.0   2.0     Al Jafer   
3                   3 2021-01-01 00:15:00   2392920.0   2.0     Al Jafer   
4                   4 2021-01-01 00:20:00   2392921.0   2.0     Al Jafer   
...               ...                 ...         ...   ...          ...   
7220790       7220790 2023-12-31 23:35:00   8467900.0  29.0  Ras Muneeef   
7220791       7220791 2023-12-31 23:40:00   8467901.0  29.0  Ras Muneeef   
7220792       7220792 2023-12-31 23:45:00   8467902.0  29.0  Ras Muneeef   
7220793       7220793 2023-12-31 23:50:00   8467903.0  29.0  Ras Muneeef   
7220794       7220794 2023-12-31 23:55:00   8467904.0  29.0  Ras Muneeef   

               lon        lat     alt  wind_speed_knot  wind_direction_degree  \
0     

In [5]:

# Define the file path for the consumption data
consumption_file_path = "/Users/computer/Downloads/Load/Load/data/consumption_data/SUB_LOAD_15MIN - 2023.xlsx"



consumption_df = pd.read_excel(consumption_file_path)

# Read the consumption data from the Excel file and Drop empty columns
consumption_df = consumption_df.dropna(axis=1, how='all')
# Display the first few rows of the DataFrame to confirm it is loaded correctly
print("Consumption Data:")
print(consumption_df.head())


# Combine date and time into a single datetime column
consumption_df['date time'] = pd.to_datetime(consumption_df['date time'].astype(str).astype(str))

# Display the resulting DataFrame
print("Combined DateTime Data:")
print(consumption_df)


Consumption Data:
  Unnamed: 1           date time Unnamed: 3          Unnamed: 4   ABDOON_P  \
0 2022-12-22 2022-12-22 00:00:00          - 2022-12-22 00:15:00  68.764967   
1        NaT 2022-12-22 00:15:00          - 2022-12-22 00:30:00  66.653231   
2        NaT 2022-12-22 00:30:00          - 2022-12-22 00:45:00  64.597852   
3        NaT 2022-12-22 00:45:00          - 2022-12-22 01:00:00  62.472460   
4        NaT 2022-12-22 01:00:00          - 2022-12-22 01:15:00  60.585692   

   MW  ALHIZAM_P MW.1    ALSALT_P MW.2  ...  MW.45   ABDALI_P    AMMANS_P  \
0      74.628408       101.051490       ...         99.267082  142.179793   
1      71.736453        96.527432       ...         95.861026  135.385285   
2      69.361243        92.596907       ...         92.176326  130.173645   
3      67.281793        88.787623       ...         88.520570  124.715073   
4      64.917022        85.571806       ...         85.197838  120.251608   

  AQABA A2_P  AQABA_TH_P   BAYADER_P  GHORSAF_P   

## Merge the weather and consumption datasets

In [7]:
# Rename the datetime columns to a common name
weather_df.rename(columns={'date_time_weather': 'datetime'}, inplace=True)
consumption_df.rename(columns={'date time': 'datetime'}, inplace=True)

print(weather_df['datetime'].head())
print(consumption_df['datetime'].head())


0   2021-01-01 00:00:00
1   2021-01-01 00:05:00
2   2021-01-01 00:10:00
3   2021-01-01 00:15:00
4   2021-01-01 00:20:00
Name: datetime, dtype: datetime64[ns]
0   2022-12-22 00:00:00
1   2022-12-22 00:15:00
2   2022-12-22 00:30:00
3   2022-12-22 00:45:00
4   2022-12-22 01:00:00
Name: datetime, dtype: datetime64[ns]


In [8]:
import pandas as pd

# Convert to datetime format
weather_df['datetime'] = pd.to_datetime(weather_df['datetime'])
consumption_df['datetime'] = pd.to_datetime(consumption_df['datetime'])

# Filter weather data to start from '2022-12-22 00:00:00'
weather_df = weather_df[weather_df['datetime'] >= '2022-12-22 00:00:00']

# Merge on the datetime column
merged_df = pd.merge(consumption_df, weather_df, on='datetime', how='inner')

# Display result
print(merged_df.head())


  Unnamed: 1   datetime Unnamed: 3          Unnamed: 4   ABDOON_P  MW  \
0 2022-12-22 2022-12-22          - 2022-12-22 00:15:00  68.764967       
1 2022-12-22 2022-12-22          - 2022-12-22 00:15:00  68.764967       
2 2022-12-22 2022-12-22          - 2022-12-22 00:15:00  68.764967       
3 2022-12-22 2022-12-22          - 2022-12-22 00:15:00  68.764967       
4 2022-12-22 2022-12-22          - 2022-12-22 00:15:00  68.764967       

   ALHIZAM_P MW.1   ALSALT_P MW.2  ...        lon        lat    alt  \
0  74.628408       101.05149       ...  36.135556  30.342222  865.0   
1  74.628408       101.05149       ...  35.493611  31.058333 -350.0   
2  74.628408       101.05149       ...  35.980556  32.488889  590.0   
3  74.628408       101.05149       ...  35.855278  32.534722  616.0   
4  74.628408       101.05149       ...  38.194722  32.503889  683.0   

  wind_speed_knot  wind_direction_degree irradiance  \
0             NaN                    NaN        NaN   
1             NaN       

In [9]:
print("Merged Data Columns:", merged_df.columns)
print(merged_df.head())


Merged Data Columns: Index(['Unnamed: 1', 'datetime', 'Unnamed: 3', 'Unnamed: 4', 'ABDOON_P', 'MW',
       'ALHIZAM_P', 'MW.1', 'ALSALT_P', 'MW.2',
       ...
       'lon', 'lat', 'alt', 'wind_speed_knot', 'wind_direction_degree',
       'irradiance', 'relative_humidity_percent', 'air_temperature_c',
       'rainfall_mm', 'irradiance_corrected'],
      dtype='object', length=117)
  Unnamed: 1   datetime Unnamed: 3          Unnamed: 4   ABDOON_P  MW  \
0 2022-12-22 2022-12-22          - 2022-12-22 00:15:00  68.764967       
1 2022-12-22 2022-12-22          - 2022-12-22 00:15:00  68.764967       
2 2022-12-22 2022-12-22          - 2022-12-22 00:15:00  68.764967       
3 2022-12-22 2022-12-22          - 2022-12-22 00:15:00  68.764967       
4 2022-12-22 2022-12-22          - 2022-12-22 00:15:00  68.764967       

   ALHIZAM_P MW.1   ALSALT_P MW.2  ...        lon        lat    alt  \
0  74.628408       101.05149       ...  36.135556  30.342222  865.0   
1  74.628408       101.05149       .

## Keep necessary columns

In [11]:
# List of columns to keep
columns_to_keep = [
    'datetime', 'ABDOON_P', 'ALHIZAM_P', 'ALSALT_P', 'AQABAIN_P',
    'ASHRFIA_P', 'ATTARATX_P', 'AZRAQ_P', 'BROADCA_P', 'C_CENTER_P',
    'DHULEIL_P', 'DISI_P', 'EL_HASA_P', 'HASHMYA_P', 'HASAN_INDU_P',
    'HUSAIN_UN_P', 'IRBID_E P', 'IRBID_N P', 'ISHTAFIN P', 'KARAK P',
    'MADABA_S P', 'MAFRAQ_P', 'MANARAH_P', 'MARQA_P', 'M_CEMENT P',
    'N_KARAK P', 'QATRANA P', 'QUWEIRA P', 'Q_CEMENT P', 'RAJIHI P',
    'RAMA P', 'RASHADI_P', 'REHAB P', 'RESHA_N P', 'REWESHED P', 'SABHA P',
    'SAFAWI P', 'SAHAB P', 'SHEDIA_P', 'SUBEIHI P', 'SWEIMEH P',
    'TAFILA_C P', 'TAREQ P', 'UNIVRSTY P', 'WAQAS P', 'ABDALI_P',
    'AMMANS_P', 'AQABA A2_P', 'AQABA_TH_P', 'BAYADER_P', 'GHORSAF_P',
    'MAAN_P', 'Q.A.I.A_P', 'ZERQA_P', 'id', 'name', 'lon', 'lat', 'alt',
    'wind_speed_knot', 'wind_direction_degree', 'irradiance',
    'relative_humidity_percent', 'air_temperature_c', 'rainfall_mm',
    'irradiance_corrected'
]

# Keep only the specified columns
merged_df = merged_df[columns_to_keep]

# Check the cleaned columns
print("Filtered Merged Data Columns:", merged_df.columns)


Filtered Merged Data Columns: Index(['datetime', 'ABDOON_P', 'ALHIZAM_P', 'ALSALT_P', 'AQABAIN_P',
       'ASHRFIA_P', 'ATTARATX_P', 'AZRAQ_P', 'BROADCA_P', 'C_CENTER_P',
       'DHULEIL_P', 'DISI_P', 'EL_HASA_P', 'HASHMYA_P', 'HASAN_INDU_P',
       'HUSAIN_UN_P', 'IRBID_E P', 'IRBID_N P', 'ISHTAFIN P', 'KARAK P',
       'MADABA_S P', 'MAFRAQ_P', 'MANARAH_P', 'MARQA_P', 'M_CEMENT P',
       'N_KARAK P', 'QATRANA P', 'QUWEIRA P', 'Q_CEMENT P', 'RAJIHI P',
       'RAMA P', 'RASHADI_P', 'REHAB P', 'RESHA_N P', 'REWESHED P', 'SABHA P',
       'SAFAWI P', 'SAHAB P', 'SHEDIA_P', 'SUBEIHI P', 'SWEIMEH P',
       'TAFILA_C P', 'TAREQ P', 'UNIVRSTY P', 'WAQAS P', 'ABDALI_P',
       'AMMANS_P', 'AQABA A2_P', 'AQABA_TH_P', 'BAYADER_P', 'GHORSAF_P',
       'MAAN_P', 'Q.A.I.A_P', 'ZERQA_P', 'id', 'name', 'lon', 'lat', 'alt',
       'wind_speed_knot', 'wind_direction_degree', 'irradiance',
       'relative_humidity_percent', 'air_temperature_c', 'rainfall_mm',
       'irradiance_corrected'],
      d

## Delete empty columns

In [13]:
# Calculate the percentage of missing values in each column
missing_percentage = merged_df.isnull().mean() * 100

# Filter for columns that have missing values (greater than 0%)
columns_with_missing_values = missing_percentage[missing_percentage > 0]

# Display columns with missing values and their percentages
print("Columns with Missing Values and Their Percentages:")
print(columns_with_missing_values)



Columns with Missing Values and Their Percentages:
wind_speed_knot              10.108386
wind_direction_degree        10.108386
irradiance                   10.108631
relative_humidity_percent    10.108386
air_temperature_c            10.113036
rainfall_mm                  10.108386
dtype: float64


## finding better imputation techniques

In [15]:
# Check the data types for the columns with missing values
missing_columns = [
    'wind_speed_knot', 'wind_direction_degree', 'irradiance',
    'relative_humidity_percent', 'air_temperature_c', 'rainfall_mm'
]

data_types_missing = merged_df[missing_columns].dtypes
print("Data Types for Columns with Missing Values:")
print(data_types_missing)


Data Types for Columns with Missing Values:
wind_speed_knot              float64
wind_direction_degree        float64
irradiance                   float64
relative_humidity_percent    float64
air_temperature_c            float64
rainfall_mm                  float64
dtype: object


In [16]:
print(type(merged_df))


<class 'pandas.core.frame.DataFrame'>


In [17]:
import pandas as pd
from sklearn.impute import KNNImputer

# Ensure merged_df is a DataFrame (already confirmed)
# Select numeric columns for KNN imputation
numeric_columns = merged_df.select_dtypes(include=['float64', 'int64']).columns

# Initialize KNN Imputer
knn_imputer = KNNImputer(n_neighbors=3)  # You can adjust the number of neighbors

# Perform KNN imputation
knn_imputed_data = knn_imputer.fit_transform(merged_df[numeric_columns])
knn_imputed_df = pd.DataFrame(knn_imputed_data, columns=numeric_columns)

# Replace original numeric columns with imputed values
merged_df[numeric_columns] = knn_imputed_df

# Check the result
print(merged_df.isnull().sum())  # Print the count of missing values in each column after imputation


KeyboardInterrupt: 

## Fill the nulls with median for the columns that still have nulls 

In [ ]:
# Columns to fill with median values
columns_to_fill = merged_df.columns[merged_df.isnull().any()]

# Fill missing values with the median for the specified columns
for column in columns_to_fill:
    median_value = merged_df[column].median()
    merged_df[column].fillna(median_value, inplace=True)

# Check if there are still missing values after filling
missing_values_count = merged_df.isnull().sum()
if (missing_values_count > 0).any():
    print("Some columns still have missing values:")
    print(missing_values_count[missing_values_count > 0])
else:
    print("All missing values have been filled with median values.")


In [ ]:
merged_df.head()